CUNY-Specific Analytics
1. Which MTA bus routes are highly utilized by CUNY students?
2. How do violation rates compare among CUNY campuses?

In [36]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path

Data loading/cleaning - CUNY campuses

In [37]:
url = "https://data.ny.gov/resource/irqs-74ez.csv"
campuses = pd.read_csv(url)
campuses.head() 

,college_or_institution_type,campus,campus_website,address,city,state,zip,lat,long,georeference
0,Community Colleges,Borough of Manhattan Community College,https://www.bmcc.cuny.edu/,199 Chambers Street,New York,NY,10007-1044,40.717367,-74.012178,POINT (-74.012178 40.717367)
1,Community Colleges,Bronx Community College,https://www.bcc.cuny.edu/,2155 University Avenue,Bronx,NY,10453-2804,40.856673,-73.910127,POINT (-73.910127 40.856673)
2,Community Colleges,Hostos Community College,http://hostos.cuny.edu,500 Grand Concourse,Bronx,NY,10451-5323,40.817828,-73.926862,POINT (-73.926862 40.817828)
3,Community Colleges,Kingsborough Community College,http://kbcc.cuny.edu,2001 Oriental Boulevard,Brooklyn,NY,11235-2333,40.578349,-73.934465,POINT (-73.934465 40.578349)
4,Community Colleges,LaGuardia Community College,https://www.laguardia.edu/,31-10 Thomson Avenue,Long Island City,NY,11101-3007,40.743951,-73.935154,POINT (-73.935154 40.743951)


In [38]:
campuses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   college_or_institution_type  26 non-null     object 
 1   campus                       26 non-null     object 
 2   campus_website               26 non-null     object 
 3   address                      26 non-null     object 
 4   city                         26 non-null     object 
 5   state                        26 non-null     object 
 6   zip                          26 non-null     object 
 7   lat                          26 non-null     float64
 8   long                         26 non-null     float64
 9   georeference                 26 non-null     object 
dtypes: float64(2), object(8)
memory usage: 2.2+ KB


In [39]:
campuses[['campus', 'lat', 'long']]

,campus,lat,long
0,Borough of Manhattan Community College,40.717367,-74.012178
1,Bronx Community College,40.856673,-73.910127
2,Hostos Community College,40.817828,-73.926862
3,Kingsborough Community College,40.578349,-73.934465
4,LaGuardia Community College,40.743951,-73.935154
5,Queensborough Community College,40.756150,-73.757550
6,Guttman Community College,40.752846,-73.984133
7,Medgar Evers College,40.666240,-73.957349
8,New York City College of Technology,40.695507,-73.987882
9,College of Staten Island,40.608648,-74.153563


In [40]:
# conversion for gpd - mapping
campuses['geometry'] = campuses.apply(lambda row: Point(row['long'], row['lat']), axis=1)
campuses_gdf = gpd.GeoDataFrame(campuses, geometry='geometry', crs="EPSG:4326")

Data loading/cleaning - Bus stops/routes

In [41]:
# GTFS dataset
GTFS_ROOT = Path("../raw_data")

feeds = {"bronx": "gtfs_bronx",
         "brooklyn": "gtfs_brooklyn",
         "manhattan": "gtfs_manhattan",
         "queens": "gtfs_queens",
         "staten_island": "gtfs_staten_island",
         "busco": "gtfs_busco"}

def load_concat(file_name):
    frames = []
    for key, sub in feeds.items():
        p = GTFS_ROOT / sub / file_name
        if p.exists():
            df = pd.read_csv(p)
            df["feed"] = key
            frames.append(df)
    if not frames:
        raise FileNotFoundError(f"Could not find {file_name} in any of the feed folders")
    return pd.concat(frames, ignore_index=True)

stops = load_concat("stops.txt")
routes = load_concat("routes.txt")
trips = load_concat("trips.txt")
stop_times = load_concat("stop_times.txt")

print(stops.shape, routes.shape, trips.shape, stop_times.shape)
stops.head()

(14456, 10) (1532, 10) (230451, 8) (7447673, 9)


,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,feed
0,100014,BEDFORD PK BLVD/GRAND CONCOURSE,NaN,40.872562,-73.888156,NaN,NaN,0.0,NaN,bronx
1,100017,PAUL AV/W 205 ST,NaN,40.876836,-73.889710,NaN,NaN,0.0,NaN,bronx
2,100018,PAUL AV/WEST MOSHOLU PKWY SOUTH,NaN,40.880392,-73.886081,NaN,NaN,0.0,NaN,bronx
3,100019,GRAND CONCOURSE/E 138 ST,NaN,40.813496,-73.929489,NaN,NaN,0.0,NaN,bronx
4,100020,GRAND CONCOURSE/E 144 ST,NaN,40.816812,-73.928001,NaN,NaN,0.0,NaN,bronx


In [42]:
# conversion for gpd - mapping
stops_gdf = gpd.GeoDataFrame(stops.assign(geometry=[Point(xy) for xy in zip(stops["stop_lon"],
            stops["stop_lat"])]),
            geometry="geometry",
            crs="EPSG:4326")

Lookup for which routes serve which stops

In [43]:
# cleaning columns presented
stop_times_small = stop_times[["trip_id", "stop_id"]]
trips_small = trips[["trip_id", "route_id"]]

stops_to_route = (stop_times_small.merge(trips_small, on="trip_id", how="inner").drop_duplicates(subset=["stop_id", "route_id"]))
stops_to_route.head()

,trip_id,stop_id,route_id
0,GH_O5-Weekday-033300_BX238_608,101734,BX26
1,GH_O5-Weekday-033300_BX238_608,101735,BX26
2,GH_O5-Weekday-033300_BX238_608,104619,BX26
3,GH_O5-Weekday-033300_BX238_608,101737,BX26
4,GH_O5-Weekday-033300_BX238_608,101738,BX26


Routes Near CUNY Campuses 
(Chose ~500m for campus buffer)

In [44]:
# conversion to meters (for buffer)
campuses_meters = campuses_gdf.to_crs(3857)
stops_meters = stops_gdf.to_crs(3857)

# buffer radius (currently 500 meters)
campus_buffers = campuses_meters.copy()
campus_buffers["geometry"] = campuses_meters.buffer(500)

# checking which stops fall inside each campus buffer
stops_near_campus = gpd.sjoin(stops_meters, campus_buffers[["campus", "geometry"]], how="inner", predicate="within")
stops_near_campus = stops_near_campus.rename(columns={"campus":"campus_name"})

# conversion back to latitude/longitude
stops_near_campus = stops_near_campus.to_crs(4326)


In [45]:
# mapping stops -> routes then aggregating routes per campus
routes_near_campus = (stops_near_campus[["stop_id", "campus_name"]]
                      .merge(stops_to_route, on="stop_id", how="left")
                      .dropna(subset=["route_id"])
                      .merge(routes[["route_id", "route_short_name", "route_long_name"]], on="route_id", how="left")
                      .drop_duplicates(subset=["campus_name", "route_id"])
                      )

# show top routes per campus
top_routes_by_campus = (stops_near_campus[["stop_id", "campus_name"]]
                         .merge(stops_to_route, on="stop_id", how="left")
                         .dropna(subset=["route_id"])
                         .groupby(["campus_name", "route_id"])
                         .size().reset_index(name="nearby_stop_count")
                         .sort_values(["campus_name", "nearby_stop_count"], ascending=[True,False])
                         )

top_routes_by_campus.head()

,campus_name,route_id,nearby_stop_count
23,Baruch College,SIM3,20
31,Baruch College,X37,19
32,Baruch College,X38,19
20,Baruch College,SIM10,18
21,Baruch College,SIM11,18


Data loading/cleaning - MTA Bus ACE Violations

In [47]:
file_path = "../raw_data/mta_ace_violations.csv"

# loading mta ace dataset in chunks (showing these columns listed)
use_columns = [
    "Violation ID", "First Occurrence", "Violation Type", "Bus Route ID", "Stop ID", "Stop Name", "Violation Latitude", "Violation Longitude"
    ]

violations_iterator = pd.read_csv(file_path, usecols=use_columns, chunksize=100000)

# putting into one dataframe with the listed columns
violations = pd.concat(violations_iterator, ignore_index=True)

print(violations.shape)
print(violations.head())

(3778568, 8)
   Violation ID        First Occurrence        Violation Type Bus Route ID  \
0     489749182  08/20/2025 11:12:08 PM       MOBILE BUS STOP         BX36   
1     489744714  08/20/2025 11:48:59 PM       MOBILE BUS STOP         BX28   
2     489743631  08/20/2025 10:33:13 PM  MOBILE DOUBLE PARKED         Q53+   
3     489741945  08/20/2025 10:50:45 PM       MOBILE BUS STOP         Q44+   
4     489741940  08/20/2025 10:52:57 AM       MOBILE BUS STOP         M101   

   Violation Latitude  Violation Longitude  Stop ID  \
0           40.840509           -73.881189   102498   
1           40.874017           -73.890646   100080   
2           40.721971           -73.867136   550473   
3           40.762529           -73.831728   501140   
4           40.815113           -73.955040   401458   

                    Stop Name  
0     EAST TREMONT AV/VYSE AV  
1   PAUL AV/BEDFORD PARK BLVD  
2  WOODHAVEN BLVD/PENELOPE AV  
3              UNION ST/35 AV  
4       AMSTERDAM AV/W 131 


Linking the datasets
- Which bus routes serve each CUNY campus
- Find the violation counts for those routes (+violation types?)
- Compare across CUNY campuses

Visualizations/Insights

Recommendations